In [1]:
# !pip install simple-elmo

In [14]:
classes = [
    'Agricolture',
    'Cross',
    'Education',
    'Food Security',
    'Health',
    'Livelihoods',
    'Logistics',
    'Nutrition',
    'Protection',
    'Shelter',
    'Wash'
]

In [2]:
from pathlib import Path
from tqdm import tqdm
import json

In [3]:
import torch

In [4]:
# from simple_elmo import ElmoModel
import pandas as pd

In [5]:
data = Path('data_prep/final_data/en/')

In [6]:
def preprocess(df):
    df.sector_ids = df.sector_ids.apply(eval)
    return df

In [15]:
def process_multiclass(df, train=False):
    relevant_train = df[df.is_relevant == 1]
    relevant_train.sector_ids = relevant_train.sector_ids.apply(eval)
    relevant_train = relevant_train[relevant_train.sector_ids.apply(len) > 0]
    relevant_train.sector_ids = relevant_train.sector_ids.apply(lambda x: ';'.join(classes[i] for i in x))
    return relevant_train    

In [16]:
raw = pd.read_csv('data_prep/data/entries_raw.csv')
train = process_multiclass(pd.read_csv(data / 'sentences_en_train.csv'))
test = process_multiclass(pd.read_csv(data / 'sentences_en_test.csv'))

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
train

,doc_id,sentence_id,sentence_text,is_relevant,sector_ids
20,51657,16,Some children remain living with a single pare...,1,Education;Livelihoods
25,51542,1,Colombia s 50- year-long armed conflict left 7...,1,Food Security
68,51542,47,Monitoring Remote monitoring results indicate ...,1,Food Security
71,51530,1,"Media report, as of 16 March, seven fatalities...",1,Cross
73,51529,0,The Mothers of Soacha discovered their sons we...,1,Protection
...,...,...,...,...,...
191852,34548,254,A high portion of export earnings from agricul...,1,Agricolture
191853,34548,255,Some Latin American countries are particularly...,1,Agricolture
191860,34548,288,"It shows that many small countries in general,...",1,Livelihoods;Logistics
191861,34548,289,"In addition to their food import dependency, t...",1,Livelihoods;Logistics


In [25]:
for sentence, sectors in zip(train.sentence_text, train.sector_ids):
    print(sentence)
    print(classes)
    print()
    input()
    print(sectors)
    print('\n\n')
    
    a = input()
    if a == 's':
        break

Some children remain living with a single parent, struggling to survive, and are often pulled out of school to earn income for the remaining family.
['Agricolture', 'Cross', 'Education', 'Food Security', 'Health', 'Livelihoods', 'Logistics', 'Nutrition', 'Protection', 'Shelter', 'Wash']


Education;Livelihoods




Colombia s 50- year-long armed conflict left 7.8 million IDPs, and while the country has set a path of peacebuilding and stabilization, violence caused by armed groups linked to illegal economies still affects thousands of Colombians every year.
['Agricolture', 'Cross', 'Education', 'Food Security', 'Health', 'Livelihoods', 'Logistics', 'Nutrition', 'Protection', 'Shelter', 'Wash']


Food Security




Monitoring Remote monitoring results indicate that WFP s beneficiaries have improved their overall food intake in 2020, as measured by the Food Consumption Score (FCS).
['Agricolture', 'Cross', 'Education', 'Food Security', 'Health', 'Livelihoods', 'Logistics', 'Nutrition', 'Pro